# Piping and List Comprehensions

## Objectives

1. Identify and use the basic structures of combinind list comprehensions and piping
2. Learn about composing list comprehension lambdas

## How we compose code that uses list comprehension

When composing code that uses list comprehensions

* Each step is (temporarily) save using assignment expression.
* The last output becomes the next input.
* Only the final result *really* needs to be saved.

In [1]:
(nums := [i for i in range(100)])[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [2]:
(sqrs := [item ** 2 for item in nums])[:10]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [3]:
(small_sqrs := [i for i in sqrs if i <= 100])

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

## Legos, or How do we can our code click!

In this activity, we will look at three sets of tools that will allow us to build less verbose and more readable code, namely

1. Using `composable.pipable` to compose each step as a pipe, and
2. Abstracting the comprehension using `map` and `filter`,

### Using `composable.pipable` and `>>` to compose with pipes.

The `pipeable` function from the `composable` package 

1. Is used to *decorate* any function created with `lambda` or `def`, and
2. Allows (forward) piping using the `>>` operator.

In [4]:
%pip install composable

Note: you may need to restart the kernel to use updated packages.


In [1]:
from composable import pipeable

In [6]:
f = pipeable(lambda x: x**2)

In [7]:
@pipeable
def g(x):
    """ Adds 2 to any number.
    """
    return x + 2

In [8]:
2 >> f >> g

6

In [9]:
2 >> g >> f
2 >> g >> f

16

### Piping in `R` versus `Python`

**Python.**

1. Functions must be pre-decorated with `pipeable`,
2. Use `>>` operator, and
3. Piped value is pushed into the **last argument.**

**R.**

1. Works out of the box with all functions,
2. Use `%>%` operator which is imported from `dplyr`/`tidyverse`, and
3. Piped value is pushed into the **first argument.**

## Step 1 - `pipeable` functions with comprehensions
    
<img src="./img/pipe_and_list_comp.png" width=600>

### Example 1 - Redoing the squares example.

In [10]:
(nums := pipeable(lambda N: [i for i in range(N)]))

<function <lambda> at 0x0000018D0FDFE840>

In [11]:
(sqrs := pipeable(lambda L: [item ** 2 for item in L]))

<function <lambda> at 0x0000018D0FDFE2A0>

In [12]:
(small_sqrs := pipeable(lambda L: [i for i in L if i <= 100]))

<function <lambda> at 0x0000018D0FDFE160>

In [13]:
(100
 >> nums
 >> sqrs
 >> small_sqrs
)[:10]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

## Example 2 - Splitting and processing string

**Task.** Compute the average length of all even-length words.

* Use `split` to cut a string into a list of strings
* Use a comprehension to process the list.

In [14]:
quote = "Don't judge each day by the harvest you reap but by the seeds that you plant."
words = quote.split(" ")
lengths =  [len(word) for word in words]
even_len = [n for n in lengths if n % 2 == 0]
mean = sum(even_len)/len(even_len)
mean

3.6666666666666665

In [15]:
split = pipeable(lambda sep, s: s.split(sep))
word_lengths = pipeable(lambda L: [len(word) for word in L])
even_len = pipeable(lambda L: [n for n in L if n % 2 == 0])
mean = pipeable(lambda L: sum(L)/len(L))

In [16]:
(quote
 >> split(" ")
 >> word_lengths
 >> even_len
 >> mean
)

3.6666666666666665

### Step 2 - Abstract the comprehension using pipeable `map` and `filter`.

<img src="./img/abstract_w_map_and_filter.png" width=600>

#### Built in `map` and `filter`

Python has `map` and `filter` by default, but

1. Lazy, and
2. No piping

In [17]:
# Run before importing (or restart kernel)
map(lambda x: x**2, range(100))

In [18]:
(range(100)
 >> map(lambda x: x**2)
)

TypeError: map() must have at least two arguments.

### Getting `pipeable` and Eager map/filter from `composable`

In [2]:
# Pipeable, eager versions of map & filter
from composable.strict import map, filter

In [20]:
(range(100)
 >> map(lambda x: x**2)
 >> filter(lambda x: x > 100)
)[:10]

[121, 144, 169, 196, 225, 256, 289, 324, 361, 400]

## Converting piped comprehensions to map/filter

### Example 1

#### Old version

In [26]:
nums = pipeable(lambda N: [i for i in range(N)])
sqrs = pipeable(lambda L: [item ** 2 for item in L])
small_sqrs = pipeable(lambda L: [i for i in L if i <= 100])

#### New version

In [22]:
# Step 1 - pipeable inner functions
make_range = pipeable(lambda N: range(N))
sqr_num = pipeable(lambda i: i **2)
at_most_100 = pipeable(lambda i: i <= 100)

In [23]:
# Step 2 - convert to a map/filter pipe

In [24]:
(100
 >> make_range
 >> map(sqr_num)
 >> filter(at_most_100)
)[:10]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

## <font color='red'> Exercise 2.3 </font>
Convert the second example to piped code

#### Old Version

In [3]:
quote = "Don't judge each day by the harvest you reap but by the seeds that you plant."
split = pipeable(lambda sep, s: s.split(sep))
word_lengths = pipeable(lambda L: [len(word) for word in L])
even_len = pipeable(lambda L: [n for n in L if n % 2 == 0])
mean = pipeable(lambda L: sum(L)/len(L))

In [4]:
(quote
 >> split(" ")
 >> word_lengths
 >> even_len
 >> mean
)

3.6666666666666665

#### New version

In [5]:
# Your new version here
plen = pipeable(lambda L: len(L))
even = pipeable(lambda n: n % 2 == 0)

In [6]:
(quote
 >> split(" ")
 >> map(plen)
 >> filter(even)
 >> mean
)

3.6666666666666665